In [1]:
# Base Data Science snippet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from tqdm import tqdm_notebook

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
"""Simulation where two agents fight for collectibles
"""

import sys
sys.path.append("../../")

from westworld.environment.grid import GridEnvironment
from westworld.agents.grid import BaseAgent,BaseObstacle,BaseTrigger,BaseCollectible
from westworld.simulation.simulation import Simulation
from westworld.colors import *

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
class Agent(BaseAgent):
    
    def init(self):
        self.score = 0
        self.target = None
    
    def step(self):
        
        # Safety check
        if self.target is not None:
            if self.target not in self.env._objects:
                self.target = None

        # Find next target
        if self.target is None:
            _,ids = self.find_closest(k = 1,condition = {"collectible":True})
            if len(ids) == 0:
                self.env.finish_episode()
            else:
                stop = False
                self.target = ids[0]
        
        if not self.env.done:
            self.move_towards(obj = self.target,n = 10)
        
        
class Collectible(BaseCollectible):
    
    collectible = True
    
    def on_collision(self,objects):
        obj = self.env[objects[0]]
        obj.score += 1
        obj.target = None
        

In [9]:
#==================================================================================================
# SIMULATION
#==================================================================================================
        
# Setup agents
agent = Agent(1,1,color = RED)
agent2 = Agent(75,25,color = BLUE)

# Setup collectibles as random spawner
collectible_spawner = lambda x,y : Collectible(x,y,color = WHITE)

# Setup environment
env = GridEnvironment(80,30,10,objects = [agent,agent2],show_grid=True,grid_color = (50,50,50))
env.spawn(collectible_spawner,100)
env.render()

# Prepare simulation
sim = Simulation(env,fps = 30,name="CollectiblesMultiplayer")

sim.run_episode(n_steps = 1000,save = True)


[INFO] Saving gif at ./captures/CollectiblesMultiplayer_1590155766.gif

